In [1]:
import io
import os
import re
import sys
import csv
import gzip
import time
import json
import pickle
import secrets
import multiprocessing
import subprocess
import resource
import pandas as pd
from scipy.stats import pearsonr
# import sqlite3
# from PIL import Image
import matplotlib.pyplot as plt
# from plotnine import *
import numpy as np
import scipy as sp
import statsmodels.api as sm
import random
# from collections import Counter
# import seaborn as sns
# import matplotlib.colors as mcolors
# from matplotlib.ticker import FuncFormatter
import itertools
import collections
import pyreadr
import psa
import pywfa
from IPython.display import display_html
# import patchworklib as pw
sys.path.append('/well/band/users/rbx225/software/lcwgsus/')
import lcwgsus
from lcwgsus.variables import *
from warnings import simplefilter
from hla_phase import *

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

pd.options.mode.chained_assignment = None

# Genuine logics changed from the main script were marked by string **X** in the original R files.
# Simplicity changes were marked by string **Y**

<Figure size 100x100 with 0 Axes>

In [7]:
gene = 'DRB1'
ipd_gen_file_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/alignments/'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
samples_to_remove = []

gamcc_hla = lcwgsus.read_hla_direct_sequencing(retain = 'fv', unique_two_field = False)
gamcc_hla = gamcc_hla[['SampleID', 'Locus', 'Two field1', 'Two field2']].reset_index(drop = True)

colnames = ['Sample ID'] + [label for g in HLA_GENES for label in [f'HLA-{g} 1', f'HLA-{g} 2']]
hlatypes = pd.DataFrame(columns = colnames)
for s in gamcc_hla['SampleID'].unique():
    tmp = gamcc_hla[gamcc_hla['SampleID'] == s]
    row = [s] + tmp[['Two field1', 'Two field2']].values.ravel().tolist()
    hlatypes.loc[len(hlatypes)] = row

reference_allele_file = '/well/band/users/rbx225/recyclable_files/hla/b38_reference_alleles.tsv'
reference_allele_ary = np.array(lcwgsus.read_tsv_as_lst(reference_allele_file))
phased_vcf = "/well/band/users/rbx225/GAMCC/results/imputation/vcfs/malariaGen_v1_b38/quilt.chr6.vcf.gz"
bam = "/well/band/users/rbx225/GAMCC/data/bams/IDT0482.bam"

In [15]:
hla_gene_information

,Name,Chr,Start,End,Strand
0,HLA-A,chr6,29942554,29945455,1
1,HLA-B,chr6,31354483,31357158,-1
2,HLA-C,chr6,31269169,31272071,-1
3,HLA-DMA,chr6,32948864,32953036,-1
4,HLA-DMB,chr6,32934971,32940807,-1
5,HLA-DOA,chr6,33006838,33009536,-1
6,HLA-DOB,chr6,32813216,32816951,-1
7,HLA-DPA1,chr6,33068650,33073570,-1
8,HLA-DPB1,chr6,33076042,33086238,1
9,HLA-DPB2,chr6,33129114,33129113,1


In [18]:
hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['Start']

14    32579091
Name: Start, dtype: int64

In [23]:
start = 32570000
end = 32590000

regstart = hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['Start'].values
regend = hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['End'].values
strand = hla_gene_information[hla_gene_information['Name'] == f'HLA-{gene}']['Strand'].values

reads_apart_max = 1000

command = f"samtools view {bam} chr6:{start}-{end}"
reads = subprocess.run(command, shell = True, capture_output = True, text = True).stdout[:-1].split('\n')

reads = [i.split('\t') for i in reads if '##' not in i]
reads = pd.DataFrame(reads).iloc[:,:11]
reads.columns = [
    "ID", "flag", "chr", "pos", "map_quality", "CIGAR", "chr_alt", "pos_alt", "insert_size", "sequence", "base_quality"
]

reads['pos'] = reads['pos'].astype(int)
reads['pos_alt'] = reads['pos_alt'].astype(int)

reads = reads[
    (reads['chr'] == 'chr6') &
    (reads['chr_alt'].isin(['chr6', '='])) &
    (reads['pos_alt'] + reads['sequence'].str.len() >= np.ones(len(reads))*(regstart - reads_apart_max)) & 
    (reads['pos_alt'] <= np.ones(len(reads))*(regend + reads_apart_max))
]

reads

,ID,flag,chr,pos,map_quality,CIGAR,chr_alt,pos_alt,insert_size,sequence,base_quality
47,A00711:753:HHCG3DSX7:4:1421:26404:7263,163,chr6,32577822,60,25M1I125M,=,32578071,408,GATAATGGGTTAAAGAAAAATAAAGCCCACTAGATGTAATTTCCTG...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFF...
49,A00711:753:HHCG3DSX7:4:2404:4969:8469,163,chr6,32577885,60,151M,=,32578028,293,ATCTTCACTGAAATACTCATTCCACACATTTTGGTTCTACCTTGTA...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...
50,A00711:753:HHCG3DSX7:4:1308:23882:27508,99,chr6,32577918,60,151M,=,32577971,204,GTTCTACCTTGTAATTCCACACCACGAGTCTCATATAAAATGAGAA...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...
53,A00711:753:HHCG3DSX7:4:1176:7979:8359,163,chr6,32578034,60,35M2D17M3I96M,=,32578106,226,TATCAACTTAATATATTTCTATAGGATTTATACTCATAGGGTTTAT...,"FFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFF..."
55,A00711:753:HHCG3DSX7:4:1176:7979:8359,83,chr6,32578106,60,106M3D45M,=,32578034,-226,ACAAGTAACATATAACTAAAAATAAGACATGCATAAAAACAAGTTT...,"FFFFFFFF:FFFFFFFFFFFFFFFFFF,FF,FFFFFFFFFFFF:FF..."
...,...,...,...,...,...,...,...,...,...,...,...
138,A00711:753:HHCG3DSX7:4:2118:10872:1188,163,chr6,32589728,40,80M1I70M,=,32589878,303,CTTCAGACACACCATGCTGGAGAAGAGGACAGGACCAGGGGCCAGA...,"F:FFFFFFFFFFF:,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF..."
140,A00711:753:HHCG3DSX7:4:1566:29197:8093,99,chr6,32589832,60,20S131M,=,32590055,318,CCACCACTCACATCCCAAACATAGGGAGGAAGTTACTAATTTCTTT...,"FF,FFFF:FFFFFFFF:::FF:FFFFFFFF:FFFFFF:F:FFFFFF..."
141,A00711:753:HHCG3DSX7:4:1604:27570:18646,163,chr6,32589859,45,103M2D48M,=,32589997,288,CTCCTGGATTGGGTAATCTCGTGTCGGAGAACCAATCAGCATCTGA...,"FFFFFFFFFFFFFF::FFFFFFF:FFFFFFFFFFFF,F:FFFFFFF..."
142,A00711:753:HHCG3DSX7:4:2118:10872:1188,83,chr6,32589878,60,84M2D67M,=,32589728,-303,CGTGTCGGAGAACCAATCAGCATCTGAGTTCAACAGCATCATCAGT...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...


In [56]:
gene = 'DRB1'
ipd_gen_file_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/alignments/'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')

ref_samples = pd.read_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/oneKG.samples', sep = ' ')
hlatypes = pd.read_csv('/well/band/users/rbx225/GAMCC/results/hla/imputation/ref_panel/auxiliary_files/20181129_HLA_types_full_1000_Genomes_Project_panel.txt', sep = '\t')
ref_samples_removed = ref_samples[~ref_samples['SAMPLE'].isin(hlatypes['Sample ID'].tolist())]
samples_to_remove = ref_samples_removed['SAMPLE'].tolist()
hlatypes = hlatypes[~hlatypes['Sample ID'].isin(samples_to_remove)].sort_values(by = 'Sample ID').reset_index(drop = True)

phased_vcf = "/well/band/users/rbx225/recyclable_files/ref_panels/oneKG/oneKG.chr6.vcf.gz"
reference_allele_file = '/well/band/users/rbx225/recyclable_files/hla/b38_reference_alleles.tsv'
reference_allele_ary = np.array(lcwgsus.read_tsv_as_lst(reference_allele_file))

# subset_samples = None

return_dict = phase_hla_on_haplotypes(gene = gene, 
                            ipd_gen_file_dir = ipd_gen_file_dir, 
                            hla_gene_information = hla_gene_information,
                            hlatypes = hlatypes,
                            phased_vcf = phased_vcf, 
                            reference_allele_ary = reference_allele_ary, 
                            read_from_QUILT = False, 
                            subset_vcf_samples = None,
                            sample_linker = None)

individual = 'NA12878'
ix = hlatypes.index[hlatypes['Sample ID'] == 'NA12878'][0]
display_indices = np.arange(10)

res = visualise_phase(gene, ix, hlatypes, return_dict)
compare_phase(display_indices, res)

# reference_exclude_samplelist_file = ''
# if reference_exclude_samplelist_file != '':
#     samples_to_remove = samples_to_remove + lcwgsus.read_tsv_as_lst(reference_exclude_samplelist_file)
# if samples_to_remove is not none, save to "hlauntyped.exclude.txt"

In [ ]:
# For GAMCC
gene = 'DRB1'
ipd_gen_file_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/alignments/'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
samples_to_remove = []

gamcc_hla = lcwgsus.read_hla_direct_sequencing(retain = 'fv', unique_two_field = False)
gamcc_hla = gamcc_hla[['SampleID', 'Locus', 'Two field1', 'Two field2']].reset_index(drop = True)

colnames = ['Sample ID'] + [label for g in HLA_GENES for label in [f'HLA-{g} 1', f'HLA-{g} 2']]
hlatypes = pd.DataFrame(columns = colnames)
for s in gamcc_hla['SampleID'].unique():
    tmp = gamcc_hla[gamcc_hla['SampleID'] == s]
    row = [s] + tmp[['Two field1', 'Two field2']].values.ravel().tolist()
    hlatypes.loc[len(hlatypes)] = row

reference_allele_file = '/well/band/users/rbx225/recyclable_files/hla/b38_reference_alleles.tsv'
reference_allele_ary = np.array(lcwgsus.read_tsv_as_lst(reference_allele_file))
phased_vcf = "/well/band/users/rbx225/GAMCC/results/imputation/vcfs/malariaGen_v1_b38/quilt.chr6.vcf.gz"

subset_vcf_samples = lcwgsus.read_tsv_as_lst('/well/band/users/rbx225/GAMCC/data/sample_tsvs/fv_gm_names.tsv')
subset_vcf_samples = ','.join(subset_vcf_samples)

sample_linker = pd.read_csv('/well/band/users/rbx225/GAMCC/' + SAMPLE_LINKER_FILE)
sample_linker = {k:v for k, v in zip(sample_linker['Sample_Name'], sample_linker['Chip_Name'])}

return_dict = phase_hla_on_haplotypes(gene = gene, 
                            ipd_gen_file_dir = ipd_gen_file_dir, 
                            hla_gene_information = hla_gene_information, 
                            hlatypes = hlatypes,
                            phased_vcf = phased_vcf, 
                            reference_allele_ary = reference_allele_ary, 
                            read_from_QUILT = True, 
                            subset_vcf_samples = subset_vcf_samples,
                            sample_linker = sample_linker)